In [ ]:
#1.Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, 
#которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением 
#столбца timestamp для данного значения userId.


In [1]:
import pandas as pd

In [25]:
df = pd.read_csv('ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [35]:
df_diff = df.groupby('userId').agg([min, max]).reset_index()

In [108]:
df_diff['diff'] = df_diff['timestamp']['max'] - df_diff['timestamp']['min']


In [114]:
df1 = df_diff[['userId', 'diff']]

In [110]:
df_pivot = df.pivot_table(index = 'userId', values = 'timestamp', columns = 'rating',  aggfunc = 'count', margins = True ).reset_index()
df_pivot.head()

rating,userId,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
0,1,NaN,2.0,NaN,7.0,3.0,4.0,1.0,3.0,NaN,NaN,20
1,2,NaN,2.0,NaN,4.0,NaN,36.0,NaN,23.0,NaN,11.0,76
2,3,NaN,NaN,NaN,1.0,3.0,18.0,9.0,11.0,4.0,5.0,51
3,4,NaN,5.0,NaN,5.0,NaN,23.0,NaN,52.0,NaN,119.0,204
4,5,NaN,NaN,1.0,NaN,3.0,3.0,27.0,42.0,19.0,5.0,100


In [115]:
df2 = df_pivot[df_pivot['All'] > 100][['All', 'userId']]
df2.head()


rating,All,userId
3,204,4
7,116,8
14,1700,15
16,363,17
18,423,19


In [119]:
df_join = df2.join(df1, how='left')

In [120]:
df_join.head()

,All,userId,"(userId, )","(diff, )"
3,204,4,4.0,203560.0
7,116,8,8.0,85187.0
14,1700,15,15.0,471393496.0
16,363,17,17.0,8053.0
18,423,19,19.0,5282.0


Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [121]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [129]:
rzd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
client_id      5 non-null int64
rzd_revenue    5 non-null int64
dtypes: int64(2)
memory usage: 208.0 bytes


In [122]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [130]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
client_id       5 non-null int64
auto_revenue    5 non-null int64
dtypes: int64(2)
memory usage: 208.0 bytes


In [123]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [124]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [134]:
df1 = rzd.merge(auto, how = 'outer')

In [135]:
df2 = df1.merge(air, how = 'outer')

In [137]:
df2.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [184]:
df_revenue = df2.pivot_table(index ='client_id', aggfunc = 'sum', margins = True )

In [180]:
df_revenue #  ВОПРОС: почему не создается столбец ALL после rzd_revenue? я пробовала columns = ['air_revenue', 'auto_revenue', 'rzd_revenue'],
#но возникает ошибка: 'str' object is not callable

,air_revenue,auto_revenue,rzd_revenue
client_id,,,
111,0.0,0.0,1093.0
112,0.0,0.0,2810.0
113,0.0,57483.0,10283.0
114,0.0,83.0,5774.0
115,81.0,912.0,981.0
116,4.0,4834.0,0.0
117,13.0,98.0,0.0
118,173.0,0.0,0.0
All,81.0,912.0,981.0


In [185]:
df_address = df2.merge(client_base, how = 'outer')

In [190]:
df_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

В данном случае можно обратить внимание на факт времени. 
Во сколько и в какой день мы получили координаты и во сколько и в какой день совершился визит или покупка. Также можно использовать наличие торговой точки от места работы или проживания клиента и насколько полученные координаты соответствуют этим данным. 